# Artificial Neural Network: Classifying Fossil Fuel Policies

**Author:** Fatih Uenal


**Purpose:** Provides a step-by-step interactive pipeline for the cleanup, modeling and storing of data. To be integrated into the ShinyApp: [Fossil Fuel Policy Tracker](https://fuenal.shinyapps.io/Fossil_Fuel_Policy_Tracker/) in the coming months. The ANN Model is based on code by [Shashi Badloe](https://link.springer.com/article/10.1007%2Fs10113-020-01677-8) modified to fit the goal of the Fossil Fuel Non-Proliferation Treaty Initiative. This tool was delveloped as part of of Fatih Uenal's Data Science Fellowship at [Faculty AI](https://faculty.ai/).

**Note:** Please do not change any folder names or directories created during intermediate steps in the pipeline. Doing so, may raise errors as the scripts look in folders with these specific names in specified relative directories.


For further details on the function of each script, please look into the code.

## 1 - Train and test-set definition
The first step in the pipeline is to define the training and testing data. The folder 'PDF_files' contains the PDF documents used as training or testing data. 

The following four folders should be in `PDF_files`:
- `Fossil-Fuel supply-side policy documents`: Training data for Fossil-Fuel supply-side policies (i.e., Moratoria)
- `Adaptation policy documents`: Training data for adaptation policies
- `Non-climate policy documents`: Training data for non-climate documents
- `Mixed policy documents`: Testing data, any PDF document(s) you want to predict on.

## 2 - Converting PDFs into text and translating into bag-of-words
To enable the machine learning algorithm to work with the text data, we need to extract the raw text from the PDF files. The conversion process may lead to alterations in the composition of the text. Therefore, some automated checks and edits to fix the most common conversion problems are applied (some issues may still persist).

The following scripts are used to convert and wrangle the PDF files:

In [ ]:
#Set working directory to the folder that holds all scripts
import os
script_folder = 'Python Scripts'
if not os.getcwd().endswith(script_folder):
    os.chdir(script_folder)
print("Current Working Directory " , os.getcwd())

First we will run the following script to convert all PDFs into raw text.

It attempts to preserve paragraph structure within text and fixes for invalid characters.

This connects to a server for PDF parsing so internet connection is required. If connection fails try re-running this.

Outputs folder is named [`parsed_files`](parsed_files), and holds raw text for every PDF file

In [ ]:
#Run script to convert all PDFs into raw text.

exec(open('pdf_parser.py').read())

The next step is to clean up the raw texts, and create eligible blocks based on paragraph size.

The following script uses tagger to determine word types and filters for useful words

The outputs folder named [`structured_files`](structured_files) contains a bag-of-words for every file in python list format

In [ ]:
#Run script to clean up the raw texts, and create eligible blocks based on paragraph size.

exec(open('text_cleanup.py').read())

## 3 - Building the SQLite database

A SQLite database is created to hold every block. This database allows for quick storing and retrieval of data and is required when working with big data.

Note that this requires a supply of metadata in 'metadata.txt'
The metadata should have a python dictionary format where the key is the filename and contents are a 
tuple of date and department like so: `pdf_filename.pdf: ('day month year', 'Department')`

In [ ]:
# Run script to create database and lots training data in Labeled_data and testing data in Unlabeled_data tables.
# Outputs 'climate.db' file in scripts folder. Will overwrite any file with the same name.

exec(open('sqlite_db.py').read())

## 4 - Training the model
The model is a simple feed forward neural network model. 

The input are the blocks and it assigns/adjusts weights of each word towards the three classes depending on how often they occur in the training data for that class or how often they co-occur in the same bag as a word strongly correlated to a class.

First, a vocabulary from training data is built. The output file ['conversion dictionary.txt'](conversion dictionary.txt) is a python dictionary where every word in the training corpus is assigned a number.

Then, the neural network and start training is generated. The generated model is stored in a folder named 'tensorflow/logdir

Finally, the stored model is used to predict on new data. Results are then stored in database.

In [ ]:
# Run script to build vocabulary from training data 

exec(open('numberizer.py').read())

In [ ]:
# Run this script to start training the neural network

exec(open('TF_classification_BW.py').read())

In [ ]:
#Run this script to predict on unseen data
#Predicts the class of blocks in 'Unlabeled_data' table in the climate.db database.
exec(open('TF_classification_predict.py').read())

# Prepare the visualizations
exec(open('document_prediction.py').read())

The pipeline is finished. 
The predictions for the 9 sample documents in the test folder [Mixed policy documents](PDF_files/Mitigation%20policy%20documents) are below.

![img](Plots/doc_pred.png) 
![img](Plots/doc_prob.png) 